In [ ]:
import boto3
import sagemaker
import pandas as pd

sess   = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

sm = boto3.Session().client(service_name='sagemaker', region_name=region)

In [ ]:
# HACK

training_job_name = 'tensorflow-training-2020-04-16-00-26-10-300'

In [ ]:
from sagemaker.tensorflow.serving import Model

# Following this example:
#    https://github.com/aws/sagemaker-python-sdk/blob/master/src/sagemaker/tensorflow/deploying_tensorflow_serving.rst#making-predictions-against-a-sagemaker-endpoint
    
# For network isolation mode:
#    If you are working in a network-isolation situation or if you don't 
#    want to install dependencies at runtime every time your endpoint 
#    starts or a batch transform job runs, you might want to put pre-downloaded 
#    dependencies under a lib directory and this directory as dependency. The container 
#    adds the modules to the Python path. Note that if both lib and requirements.txt are
#    present in the model archive, the requirements.txt is ignored:


# Used for multi-model
env = {
  'SAGEMAKER_TFS_DEFAULT_MODEL_NAME': 'main'
}

model = Model(entry_point='inference.py',
              source_dir='src_inference',
              model_data='s3://{}/{}/output/model.tar.gz'.format(bucket, training_job_name),
              role=role,
              framework_version="2.1.0",
              env=env)

In [ ]:
predictor = model.deploy(initial_instance_count=1, 
                         instance_type='ml.c5.xlarge')

# Load the Model

In [ ]:
transformer_model_dir = '{}/transformers/fine-tuned/'.format(models_dir)

!ls -al $transformer_model_dir

In [ ]:
cat $transformer_model_dir/config.json

In [ ]:
import os
from transformers import TFDistilBertForSequenceClassification

loaded_model = TFDistilBertForSequenceClassification.from_pretrained(transformer_model_dir,
                                                                     id2label={
                                                                       0: 1,
                                                                       1: 2,
                                                                       2: 3,
                                                                       3: 4,
                                                                       4: 5
                                                                     },
                                                                     label2id={
                                                                       1: 0,
                                                                       2: 1,
                                                                       3: 2,
                                                                       4: 3,
                                                                       5: 4
                                                                     })

In [ ]:
from transformers import DistilBertTokenizer

tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

inference_device = -1 # CPU: -1, GPU: 0
print('inference_device {}'.format(inference_device))

In [ ]:
from transformers import TextClassificationPipeline

inference_pipeline = TextClassificationPipeline(model=loaded_model, 
                                                tokenizer=tokenizer,
                                                framework='tf',
                                                device=inference_device) # -1 is CPU, >= 0 is GPU

In [ ]:
review = """I loved it!  I will recommend this to everyone."""
print(review, inference_pipeline(review))

In [ ]:
review = """Really bad.  I hope they don't make this anymore."""
print(review, inference_pipeline(review))

# Uncomment to Start Tensorboard
Let's start Tensorboard and point to the tensorboard logs that we downloaded from S3 directly.

Note:  If you pointed Tensorboard to S3 directly, you must prepend this command with `S3_REGION=[your-region]`.

In [ ]:
outputs_dir = './outputs'

In [ ]:
!aws s3 cp --recursive s3://$bucket/$training_job_name/debug-output/events/ $outputs_dir/events/

In [ ]:
!ls -al $outputs_dir/events

In [ ]:
#!S3_REGION=us-east-1 tensorboard --port 6006 --logdir s3://$bucket/$training_job_name/debug-output/events/  # <== MAKE SURE YOU INCLUDE THE TRAILING `/`
!tensorboard --port 6006 --logdir $outputs_dir/events/ # <== MAKE SURE YOU INCLUDE THE TRAILING `/`

While Tensorboard is running locally on your SageMaker Notebook instance, it is reading the training logs from Amazon S3.

Navigate to https://workshop.notebook.[your-region].sagemaker.aws/proxy/6006/  <== MAKE SURE YOU INCLUDE THE TRAILING SLASH

_Note:  Make sure you copy the trailing `/` in the link above.  If you see no data, you are likely not using the correct S3 bucket above._

![Tensorboard](img/tensorboard.png)

# Stop Tensorboard
Once you are done, hit `Kernel => Stop` to stop the running `Tensorboard` process in this notebook.